<a href="https://colab.research.google.com/github/hadil-karous/CS229_assignments/blob/main/yolov11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Install the ultralytics package
!pip install ultralytics -q

# Mount your Google Drive
from google.colab import drive
drive.mount('/content/drive')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 18.3 MB/s eta 0:00:00
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import yaml
import os

# Define your dataset path based on the screenshot
dataset_path = '/content/drive/MyDrive/yolov11/hadil'
yaml_path = os.path.join(dataset_path, 'data.yaml')

# Load the existing yaml
with open(yaml_path, 'r') as file:
    data = yaml.safe_load(file)

# Update paths to point directly to your Google Drive folders
data['train'] = os.path.join(dataset_path, 'train/images')
data['val'] = os.path.join(dataset_path, 'valid/images')
data['test'] = os.path.join(dataset_path, 'test/images')

# Save the updated yaml
with open(yaml_path, 'w') as file:
    yaml.dump(data, file, sort_keys=False)

print("✅ data.yaml successfully updated with Google Drive paths!")

✅ data.yaml successfully updated with Google Drive paths!


In [4]:
from ultralytics import YOLO

# Load the YOLO11 nano model
model = YOLO("yolo11n.pt")

# Train the model
results = model.train(
    data=yaml_path,       # Uses your newly updated data.yaml
    epochs=200,           # Number of training epochs
    imgsz=640,            # Image size
    batch=16,             # Batch size
    patience=50,          # Early stopping if no improvement for 50 epochs
    project=f"{dataset_path}/runs", # SAVES DIRECTLY TO YOUR DRIVE
    name="solar_anomaly_model",

    # Built-in augmentations
    hsv_h=0.015,
    hsv_s=0.7,
    hsv_v=0.4,
    degrees=10.0,
    translate=0.1,
    scale=0.5,
    fliplr=0.5,
    mosaic=1.0,
)

print("✅ Training Complete! Weights saved to your Google Drive.")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.4.16 🚀 Python-3.12.12 torch-2.10.0+cpu CPU (Intel Xeon CPU @ 2.20GHz)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/yolov11/hadil/data.yaml, degrees=10.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=200, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v

KeyboardInterrupt: 

In [5]:
# Load your best trained model from your Drive
best_model_path = f"{dataset_path}/runs/solar_anomaly_model/weights/best.pt"
best_model = YOLO(best_model_path)

print("\n--- Running Evaluation on TEST Set ---")
# Evaluate model performance on the test set
metrics = best_model.val(data=yaml_path, split='test')

# Print core metrics clearly
print("\n--- YOLO11 Model Metrics Analysis ---")
print(f"mAP50 (Mean Average Precision @ 50% IoU): {metrics.box.map50:.4f}")
print(f"mAP50-95 (Mean Average Precision @ 50-95% IoU): {metrics.box.map:.4f}")


--- Running Evaluation on TEST Set ---
Ultralytics 8.4.16 🚀 Python-3.12.12 torch-2.10.0+cpu CPU (Intel Xeon CPU @ 2.20GHz)
YOLO11n summary (fused): 101 layers, 2,582,932 parameters, 0 gradients, 6.3 GFLOPs
val: Fast image access ✅ (ping: 0.9±0.3 ms, read: 0.1±0.0 MB/s, size: 42.5 KB)
val: Scanning /content/drive/MyDrive/yolov11/hadil/test/labels... 21 images, 2 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 21/21 1.4s/it 29.0s
val: New cache created: /content/drive/MyDrive/yolov11/hadil/test/labels.cache
WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 42, len(boxes) = 64. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 2/2 2.6s/it 5.2s
                   all         21         64      0.639      0.746      0.749      0.516
         D